# Xavier Deep network version

# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.2028, train_loss: 2.00766087, train_accuracy: 0.4609, test_Accuracy: 0.2505
Epoch: [ 0] [    1/  468] time: 1.3025, train_loss: 1.95650053, train_accuracy: 0.3438, test_Accuracy: 0.3380
Epoch: [ 0] [    2/  468] time: 1.3983, train_loss: 1.74339414, train_accuracy: 0.5938, test_Accuracy: 0.5213
Epoch: [ 0] [    3/  468] time: 1.5040, train_loss: 1.40736580, train_accuracy: 0.7031, test_Accuracy: 0.6155
Epoch: [ 0] [    4/  468] time: 1.6017, train_loss: 1.22974730, train_accuracy: 0.7578, test_Accuracy: 0.6654
Epoch: [ 0] [    5/  468] time: 1.7014, train_loss: 1.04806364, train_accuracy: 0.7266, test_Accuracy: 0.7254
Epoch: [ 0] [    6/  468] time: 1.7972, train_loss: 0.83519191, train_accuracy: 0.7891, test_Accuracy: 0.7614
Epoch: [ 0] [    7/  468] time: 1.9029, train_loss: 0.65620446, train_accuracy: 0.7734, test_Accuracy: 0.7440
Epoch: [ 0] [    8/  468] time: 1.9987,

Epoch: [ 0] [   73/  468] time: 8.4823, train_loss: 0.09207506, train_accuracy: 1.0000, test_Accuracy: 0.9320
Epoch: [ 0] [   74/  468] time: 8.5830, train_loss: 0.18656935, train_accuracy: 0.9375, test_Accuracy: 0.9302
Epoch: [ 0] [   75/  468] time: 8.6788, train_loss: 0.29487821, train_accuracy: 0.9062, test_Accuracy: 0.9298
Epoch: [ 0] [   76/  468] time: 8.7795, train_loss: 0.12409885, train_accuracy: 0.9688, test_Accuracy: 0.9311
Epoch: [ 0] [   77/  468] time: 8.8773, train_loss: 0.21378641, train_accuracy: 0.9453, test_Accuracy: 0.9325
Epoch: [ 0] [   78/  468] time: 8.9780, train_loss: 0.19676015, train_accuracy: 0.9297, test_Accuracy: 0.9325
Epoch: [ 0] [   79/  468] time: 9.0747, train_loss: 0.20444177, train_accuracy: 0.9453, test_Accuracy: 0.9342
Epoch: [ 0] [   80/  468] time: 9.1775, train_loss: 0.22085315, train_accuracy: 0.9531, test_Accuracy: 0.9325
Epoch: [ 0] [   81/  468] time: 9.2742, train_loss: 0.12085280, train_accuracy: 0.9766, test_Accuracy: 0.9345
Epoch: [ 0

Epoch: [ 0] [  148/  468] time: 16.0571, train_loss: 0.15168709, train_accuracy: 0.9531, test_Accuracy: 0.9447
Epoch: [ 0] [  149/  468] time: 16.1578, train_loss: 0.18956207, train_accuracy: 0.9219, test_Accuracy: 0.9478
Epoch: [ 0] [  150/  468] time: 16.2635, train_loss: 0.14362098, train_accuracy: 0.9688, test_Accuracy: 0.9476
Epoch: [ 0] [  151/  468] time: 16.3662, train_loss: 0.18111187, train_accuracy: 0.9609, test_Accuracy: 0.9461
Epoch: [ 0] [  152/  468] time: 16.4660, train_loss: 0.21980138, train_accuracy: 0.9375, test_Accuracy: 0.9451
Epoch: [ 0] [  153/  468] time: 16.5767, train_loss: 0.22508705, train_accuracy: 0.9297, test_Accuracy: 0.9466
Epoch: [ 0] [  154/  468] time: 16.6794, train_loss: 0.08678500, train_accuracy: 0.9766, test_Accuracy: 0.9507
Epoch: [ 0] [  155/  468] time: 16.7771, train_loss: 0.09363004, train_accuracy: 0.9844, test_Accuracy: 0.9505
Epoch: [ 0] [  156/  468] time: 16.8759, train_loss: 0.07665386, train_accuracy: 0.9766, test_Accuracy: 0.9470
E

Epoch: [ 0] [  222/  468] time: 23.5680, train_loss: 0.18299522, train_accuracy: 0.9375, test_Accuracy: 0.9569
Epoch: [ 0] [  223/  468] time: 23.6747, train_loss: 0.13780501, train_accuracy: 0.9844, test_Accuracy: 0.9560
Epoch: [ 0] [  224/  468] time: 23.7784, train_loss: 0.10430688, train_accuracy: 0.9844, test_Accuracy: 0.9468
Epoch: [ 0] [  225/  468] time: 23.8791, train_loss: 0.14654604, train_accuracy: 0.9453, test_Accuracy: 0.9385
Epoch: [ 0] [  226/  468] time: 23.9829, train_loss: 0.12154939, train_accuracy: 0.9609, test_Accuracy: 0.9457
Epoch: [ 0] [  227/  468] time: 24.0866, train_loss: 0.12093730, train_accuracy: 0.9609, test_Accuracy: 0.9527
Epoch: [ 0] [  228/  468] time: 24.1833, train_loss: 0.12300019, train_accuracy: 0.9531, test_Accuracy: 0.9559
Epoch: [ 0] [  229/  468] time: 24.2870, train_loss: 0.20082851, train_accuracy: 0.9375, test_Accuracy: 0.9530
Epoch: [ 0] [  230/  468] time: 24.3888, train_loss: 0.11232708, train_accuracy: 0.9688, test_Accuracy: 0.9455
E

Epoch: [ 0] [  297/  468] time: 31.1976, train_loss: 0.10694516, train_accuracy: 0.9609, test_Accuracy: 0.9608
Epoch: [ 0] [  298/  468] time: 31.3013, train_loss: 0.08464111, train_accuracy: 0.9766, test_Accuracy: 0.9589
Epoch: [ 0] [  299/  468] time: 31.4070, train_loss: 0.05673713, train_accuracy: 0.9844, test_Accuracy: 0.9559
Epoch: [ 0] [  300/  468] time: 31.5257, train_loss: 0.08968344, train_accuracy: 0.9766, test_Accuracy: 0.9552
Epoch: [ 0] [  301/  468] time: 31.6344, train_loss: 0.11012803, train_accuracy: 0.9688, test_Accuracy: 0.9558
Epoch: [ 0] [  302/  468] time: 31.7381, train_loss: 0.11364745, train_accuracy: 0.9609, test_Accuracy: 0.9580
Epoch: [ 0] [  303/  468] time: 31.8428, train_loss: 0.06508617, train_accuracy: 0.9766, test_Accuracy: 0.9600
Epoch: [ 0] [  304/  468] time: 31.9515, train_loss: 0.08531612, train_accuracy: 0.9688, test_Accuracy: 0.9583
Epoch: [ 0] [  305/  468] time: 32.0473, train_loss: 0.07109861, train_accuracy: 0.9766, test_Accuracy: 0.9543
E

Epoch: [ 0] [  371/  468] time: 38.6915, train_loss: 0.14539418, train_accuracy: 0.9531, test_Accuracy: 0.9617
Epoch: [ 0] [  372/  468] time: 38.7952, train_loss: 0.08836203, train_accuracy: 0.9766, test_Accuracy: 0.9630
Epoch: [ 0] [  373/  468] time: 38.8980, train_loss: 0.10843980, train_accuracy: 0.9844, test_Accuracy: 0.9645
Epoch: [ 0] [  374/  468] time: 38.9967, train_loss: 0.05740925, train_accuracy: 0.9844, test_Accuracy: 0.9658
Epoch: [ 0] [  375/  468] time: 39.0984, train_loss: 0.07275130, train_accuracy: 0.9766, test_Accuracy: 0.9660
Epoch: [ 0] [  376/  468] time: 39.2002, train_loss: 0.04190675, train_accuracy: 0.9922, test_Accuracy: 0.9658
Epoch: [ 0] [  377/  468] time: 39.3019, train_loss: 0.03691491, train_accuracy: 1.0000, test_Accuracy: 0.9641
Epoch: [ 0] [  378/  468] time: 39.4006, train_loss: 0.04690970, train_accuracy: 0.9922, test_Accuracy: 0.9626
Epoch: [ 0] [  379/  468] time: 39.4964, train_loss: 0.13970774, train_accuracy: 0.9609, test_Accuracy: 0.9631
E

Epoch: [ 0] [  446/  468] time: 46.1964, train_loss: 0.07705867, train_accuracy: 0.9922, test_Accuracy: 0.9637
Epoch: [ 0] [  447/  468] time: 46.2992, train_loss: 0.07447495, train_accuracy: 0.9766, test_Accuracy: 0.9622
Epoch: [ 0] [  448/  468] time: 46.3989, train_loss: 0.04013775, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [  449/  468] time: 46.4986, train_loss: 0.05359505, train_accuracy: 0.9766, test_Accuracy: 0.9631
Epoch: [ 0] [  450/  468] time: 46.5994, train_loss: 0.04226441, train_accuracy: 0.9766, test_Accuracy: 0.9659
Epoch: [ 0] [  451/  468] time: 46.6971, train_loss: 0.05405059, train_accuracy: 0.9844, test_Accuracy: 0.9649
Epoch: [ 0] [  452/  468] time: 46.7928, train_loss: 0.03961316, train_accuracy: 0.9844, test_Accuracy: 0.9653
Epoch: [ 0] [  453/  468] time: 46.8926, train_loss: 0.10678113, train_accuracy: 0.9766, test_Accuracy: 0.9652
Epoch: [ 0] [  454/  468] time: 47.0013, train_loss: 0.06582948, train_accuracy: 0.9688, test_Accuracy: 0.9644
E

# Test accuracy : 96.88 %